Code here is adapted from [this Colab notebook](https://colab.research.google.com/github/ultralytics/ultralytics/blob/main/examples/tutorial.ipynb#scrollTo=ktegpM42AooT)

In [52]:
import os


THIS_PATH = os.path.dirname(os.path.abspath(os.getcwd()))

DATA_PATH = os.path.join(THIS_PATH, 'data')

RAW_IMAGES_PATH = os.path.join(DATA_PATH, 'raw')
RAW_IMAGE_FILES = os.listdir(RAW_IMAGES_PATH)

ANNOTATIONS_DIR = os.path.join(DATA_PATH, "moves_vs_headers_annotations")
ANNOTATION_FILES = [f for f in os.listdir(ANNOTATIONS_DIR) if os.path.splitext(f)[0].isdigit() and os.path.splitext(f)[1].lower() == '.txt']
ANNOTATION_CLASSES_JSON_FILE = os.path.join(ANNOTATIONS_DIR, "classes.txt")

if len(RAW_IMAGE_FILES) < len(ANNOTATION_FILES):
    print(
        'Warning: There are more annotation files than images,'
        'something might be wrong with the data?'
    )

NUM_TRAINING_SAMPLES = min(len(RAW_IMAGE_FILES), len(ANNOTATION_FILES))

In [64]:
# Create a dataset YAML file for fine-tuning YOLOv8 on custom data
CONFIG_YAML_FILE = "moves_vs_headers_training_config.yaml"

# Read the classes from the annotations dir. Each line in the `.txt` files
# with bounding box annotations starts with a single int that indicates the
# index of the class in the list in `classes.txt`
with open(ANNOTATION_CLASSES_JSON_FILE, "r") as f:
    classes = [line.strip() for line in f.readlines()]

dataset_yaml_content = f"""\
path: ../data/ultralytics_yolo_format
train: images/train
val: images/val

names:
{"\n".join(f"  {i}: {c}" for i, c in enumerate(classes))}\
"""

with open(CONFIG_YAML_FILE, "w") as f:
    f.write(dataset_yaml_content)

In [58]:
# Clone the raw data files into the ultralytics yolo format with train/val split
import os
import shutil
import random


OUTPUT_DATA_DIR = os.path.join(DATA_PATH, "ultralytics_yolo_format")

OUTPUT_IMAGES_DIR = os.path.join(OUTPUT_DATA_DIR, "images")
OUTPUT_LABELS_DIR = os.path.join(OUTPUT_DATA_DIR, "labels")

TRAIN_IMAGES_DIR = os.path.join(OUTPUT_IMAGES_DIR, "train")
VAL_IMAGES_DIR = os.path.join(OUTPUT_IMAGES_DIR, "val")
TRAIN_LABELS_DIR = os.path.join(OUTPUT_LABELS_DIR, "train")
VAL_LABELS_DIR = os.path.join(OUTPUT_LABELS_DIR, "val")


# Delete and recreate the output dirs
if os.path.exists(OUTPUT_DATA_DIR):
    shutil.rmtree(OUTPUT_DATA_DIR)

os.makedirs(TRAIN_IMAGES_DIR)
os.makedirs(VAL_IMAGES_DIR)
os.makedirs(TRAIN_LABELS_DIR)
os.makedirs(VAL_LABELS_DIR)

# Split the data into train and val
TRAIN_VAL_SPLIT = 0.8
train_size = int(TRAIN_VAL_SPLIT * NUM_TRAINING_SAMPLES)
val_size = NUM_TRAINING_SAMPLES - train_size

image_annotation_pairs = list(zip(RAW_IMAGE_FILES, ANNOTATION_FILES))
random.seed(0)
random.shuffle(image_annotation_pairs)

train_files = image_annotation_pairs[:train_size]
val_files = image_annotation_pairs[train_size:]
train_files.sort()
val_files.sort()

'''Make symlinks to the images and labels'''
# Train set
for image_file, annotation_file in train_files:
    src_image = os.path.join(RAW_IMAGES_PATH, image_file)
    src_label = os.path.join(ANNOTATIONS_DIR, annotation_file)

    dst_image = os.path.join(TRAIN_IMAGES_DIR, image_file)
    dst_label = os.path.join(TRAIN_LABELS_DIR, annotation_file)

    os.symlink(src_image, dst_image)
    os.symlink(src_label, dst_label)

# Val set
for image_file, annotation_file in val_files:
    src_image = os.path.join(RAW_IMAGES_PATH, image_file)
    src_label = os.path.join(ANNOTATIONS_DIR, annotation_file)

    dst_image = os.path.join(VAL_IMAGES_DIR, image_file)
    dst_label = os.path.join(VAL_LABELS_DIR, annotation_file)

    os.symlink(src_image, dst_image)
    os.symlink(src_label, dst_label)

In [15]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.66  Python-3.12.4 torch-2.3.1+cpu CPU (Intel Core(TM) i5-9300H 2.40GHz)
Setup complete  (8 CPUs, 39.8 GB RAM, 868.9/929.7 GB disk)


In [16]:
import comet_ml
comet_ml.login()

In [62]:
from ultralytics import YOLO

# This starts from a trained model - it will download
# the weights file (.pt) automatically
model = YOLO('yolov8n.pt')
model

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

In [ ]:
# Run training. Might take a while
# 
# Note: This worked great for a larger dataset of 2000 train / 500 val samples
# and 30 epochs. Didn't really work at all on this dataset with 20 images and 50 epochs
# (did not detect any bounding boxes *at all* on val images)

model.train(data=CONFIG_YAML_FILE, epochs=3, imgsz=640)

In [60]:

RUNS_DIR = os.path.join(THIS_PATH, "runs/detect")

# Change to whatever path you want
TRAINED_MODEL_WEIGHTS_PATH = os.path.join(RUNS_DIR, "train10/weights/best.pt")


In [ ]:
# Optional, show some stats about the model's performance
# on the validation set
from ultralytics import YOLO

model = YOLO(os.path.join(TRAINED_MODEL_WEIGHTS_PATH))
model.val(data="fine_tune_yolov8_config.yaml")

In [1]:
import os
from ultralytics import YOLO
import random


# Change to whatever path you want
TRAINED_MODEL_WEIGHTS_PATH = os.path.join(RUNS_DIR, 'train10/weights/best')

model = YOLO(TRAINED_MODEL_WEIGHTS_PATH)

val_images_dir = VAL_IMAGES_DIR
val_images = os.listdir(val_images_dir)

# Choose an image idx from the validation set,
# or set to `None` to choose a random one
IMAGE_IDX: int | None = None
if IMAGE_IDX is None:
    IMAGE_IDX = random.randint(0, len(val_images) - 1)

result = model(os.path.join(val_images_dir, val_images[IMAGE_IDX]))
result[0].show()


image 1/1 c:\Users\jacks\OneDrive\Documents\Code\chess-scoresheet-2-pgn\fine_tuning\..\data\ultralytics_yolo_format\images\val\0017.png: 512x640 (no detections), 284.8ms
Speed: 12.0ms preprocess, 284.8ms inference, 8.0ms postprocess per image at shape (1, 3, 512, 640)
